# Working with feature columns

In [8]:
import pandas as pd
import tensorflow as tf
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
dataset_path = tf.keras.utils.get_file("auto-mpg.data", 
                                       ("http://archive.ics.uci.edu/ml/machine-learning-databases"
                                        "/auto-mpg/auto-mpg.data"))

column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower',
                'Weight', 'Acceleration', 'ModelYear', 'Origin']

df = pd.read_csv(dataset_path, names=column_names,
                 na_values = "?", comment='\t',
                 sep=" ", skipinitialspace=True)

df.tail()

print(df.isna().sum())

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
ModelYear       0
Origin          0
dtype: int64


In [10]:
df = df.dropna()
df = df.reset_index(drop=True)
df.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
387,27.0,4,140.0,86.0,2790.0,15.6,82,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,2
389,32.0,4,135.0,84.0,2295.0,11.6,82,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,1
391,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [11]:
df_train, df_test = sklearn.model_selection.train_test_split(df, train_size=0.8)
train_stats = df_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,313.0,23.408626,7.838892,9.0,17.0,22.3,29.0,46.6
Cylinders,313.0,5.501597,1.709934,3.0,4.0,4.0,8.0,8.0
Displacement,313.0,195.563898,104.888872,68.0,105.0,151.0,302.0,455.0
Horsepower,313.0,104.130990,38.828916,46.0,75.0,94.0,125.0,230.0
Weight,313.0,2991.958466,850.081988,1613.0,2230.0,2835.0,3630.0,4997.0
Acceleration,313.0,15.639936,2.735164,8.0,14.0,15.5,17.3,24.8
ModelYear,313.0,75.923323,3.650676,70.0,73.0,76.0,79.0,82.0
Origin,313.0,1.597444,0.826791,1.0,1.0,1.0,2.0,3.0


In [12]:
numeric_column_names = ['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']

df_train_norm, df_test_norm = df_train.copy(), df_test.copy()

for col_name in numeric_column_names:
    mean = train_stats.loc[col_name, 'mean']
    std  = train_stats.loc[col_name, 'std']
    df_train_norm.loc[:, col_name] = (df_train_norm.loc[:, col_name] - mean)/std
    df_test_norm.loc[:, col_name] = (df_test_norm.loc[:, col_name] - mean)/std
    
df_train_norm.tail()


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
13,14.0,1.461110,2.473438,3.112861,0.110626,-2.062010,70,1
156,16.0,1.461110,1.167294,1.181311,1.771643,-0.416771,75,1
173,29.0,-0.878161,-1.006436,-0.879010,-1.241008,-0.599575,75,2
336,25.8,-0.878161,-0.377198,-0.312422,-0.437556,-0.453331,81,1
314,19.1,0.291475,0.280641,-0.363930,0.457652,1.118786,80,1


In [13]:
numeric_features = []

for col_name in numeric_column_names:
    numeric_features.append(tf.feature_column.numeric_column(key=col_name))
    
numeric_features

[NumericColumn(key='Cylinders', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Displacement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Horsepower', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Weight', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Acceleration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

Next, let's group the rather fine-grained model year information into buckets to simplify the learning task for the model that we are going to train later. Concretely, we are going to assign each car into one of four "year" buckets

Note that the chosen intervals were selected arbitrarily to illustrate the concepts
of "bucketing." In order to group the cars into these buckets, we will first define
a numeric feature based on each original model year. Then, these numeric features will be passed to the bucketized_column function for which we will specify three interval cut-off values: [73, 76, 79]. The specified values include the right cut-off value. These cut-off values are used to specify half-closed intervals, for instance, (−∞, 73), [73, 76), [76, 79), and [79, ∞).

In [15]:
feature_year = tf.feature_column.numeric_column(key="ModelYear")

bucketized_features = []

bucketized_features.append(tf.feature_column.bucketized_column(
    source_column=feature_year,
    boundaries=[73, 76, 79]))

print(bucketized_features)

[BucketizedColumn(source_column=NumericColumn(key='ModelYear', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(73, 76, 79))]


In [16]:
feature_origin = tf.feature_column.categorical_column_with_vocabulary_list(
    key='Origin',
    vocabulary_list=[1, 2, 3])

categorical_indicator_features = []
categorical_indicator_features.append(tf.feature_column.indicator_column(feature_origin))

print(categorical_indicator_features)

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Origin', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0))]


# Machine learning with pre-made Estimators

In [20]:
def train_input_fn(df_train, batch_size=8):
    df = df_train.copy()
    train_x, train_y = df, df.pop('MPG')
    dataset = tf.data.Dataset.from_tensor_slices((dict(train_x), train_y))

    # shuffle, repeat, and batch the examples
    return dataset.shuffle(1000).repeat().batch(batch_size)

## inspection
ds = train_input_fn(df_train_norm)
batch = next(iter(ds))
print('Keys:', batch[0].keys())
print('Batch Model Years:', batch[0]['ModelYear'])

Keys: dict_keys(['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'ModelYear', 'Origin'])
Batch Model Years: tf.Tensor([70 80 78 76 76 76 75 82], shape=(8,), dtype=int32)


In [21]:
def eval_input_fn(df_test, batch_size=8):
    df = df_test.copy()
    test_x, test_y = df, df.pop('MPG')
    dataset = tf.data.Dataset.from_tensor_slices((dict(test_x), test_y))
    return dataset.batch(batch_size)

In [22]:
all_feature_columns = (numeric_features + 
                       bucketized_features + 
                       categorical_indicator_features)

print(all_feature_columns)

[NumericColumn(key='Cylinders', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Displacement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Horsepower', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Weight', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Acceleration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), BucketizedColumn(source_column=NumericColumn(key='ModelYear', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(73, 76, 79)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Origin', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0))]


In [23]:
regressor = tf.estimator.DNNRegressor(
    feature_columns=all_feature_columns,
    hidden_units=[32, 10],
    model_dir='models/autompg-dnnregressor/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/autompg-dnnregressor/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a43999950>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [24]:
EPOCHS = 1000
BATCH_SIZE = 8
total_steps = EPOCHS * int(np.ceil(len(df_train) / BATCH_SIZE))
print('Training Steps:', total_steps)

Training Steps: 40000


In [25]:
regressor.train(
    input_fn=lambda:train_input_fn(df_train_norm, batch_size=BATCH_SIZE),
    steps=total_steps)


reloaded_regressor = tf.estimator.DNNRegressor(
    feature_columns=all_feature_columns,
    hidden_units=[32, 10],
    warm_start_from='models/autompg-dnnregressor/',
    model_dir='models/autompg-dnnregressor/')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs ins

INFO:tensorflow:loss = 319.48804, step = 4400 (0.232 sec)
INFO:tensorflow:global_step/sec: 437.401
INFO:tensorflow:loss = 427.24686, step = 4500 (0.228 sec)
INFO:tensorflow:global_step/sec: 424.038
INFO:tensorflow:loss = 309.9594, step = 4600 (0.236 sec)
INFO:tensorflow:global_step/sec: 438.422
INFO:tensorflow:loss = 240.05905, step = 4700 (0.228 sec)
INFO:tensorflow:global_step/sec: 351.75
INFO:tensorflow:loss = 323.84912, step = 4800 (0.285 sec)
INFO:tensorflow:global_step/sec: 346.722
INFO:tensorflow:loss = 465.80463, step = 4900 (0.289 sec)
INFO:tensorflow:global_step/sec: 442.909
INFO:tensorflow:loss = 457.72784, step = 5000 (0.225 sec)
INFO:tensorflow:global_step/sec: 441.106
INFO:tensorflow:loss = 314.5749, step = 5100 (0.227 sec)
INFO:tensorflow:global_step/sec: 448.31
INFO:tensorflow:loss = 302.48874, step = 5200 (0.223 sec)
INFO:tensorflow:global_step/sec: 441.46
INFO:tensorflow:loss = 292.32645, step = 5300 (0.226 sec)
INFO:tensorflow:global_step/sec: 424.14
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 429.211
INFO:tensorflow:loss = 139.74365, step = 12500 (0.233 sec)
INFO:tensorflow:global_step/sec: 432.191
INFO:tensorflow:loss = 133.00508, step = 12600 (0.231 sec)
INFO:tensorflow:global_step/sec: 448.463
INFO:tensorflow:loss = 46.438297, step = 12700 (0.223 sec)
INFO:tensorflow:global_step/sec: 439.57
INFO:tensorflow:loss = 63.7254, step = 12800 (0.227 sec)
INFO:tensorflow:global_step/sec: 445.07
INFO:tensorflow:loss = 74.85186, step = 12900 (0.224 sec)
INFO:tensorflow:global_step/sec: 378.742
INFO:tensorflow:loss = 101.27356, step = 13000 (0.265 sec)
INFO:tensorflow:global_step/sec: 420.856
INFO:tensorflow:loss = 54.312584, step = 13100 (0.237 sec)
INFO:tensorflow:global_step/sec: 388.703
INFO:tensorflow:loss = 99.38988, step = 13200 (0.256 sec)
INFO:tensorflow:global_step/sec: 445.251
INFO:tensorflow:loss = 85.661026, step = 13300 (0.225 sec)
INFO:tensorflow:global_step/sec: 408.863
INFO:tensorflow:loss = 92.353645, step = 13400 (0.245 sec)
INFO:t

INFO:tensorflow:loss = 47.316254, step = 20700 (0.344 sec)
INFO:tensorflow:global_step/sec: 281.896
INFO:tensorflow:loss = 34.33013, step = 20800 (0.355 sec)
INFO:tensorflow:global_step/sec: 405.075
INFO:tensorflow:loss = 16.047604, step = 20900 (0.247 sec)
INFO:tensorflow:global_step/sec: 340.275
INFO:tensorflow:loss = 81.456085, step = 21000 (0.295 sec)
INFO:tensorflow:global_step/sec: 367.697
INFO:tensorflow:loss = 14.863405, step = 21100 (0.271 sec)
INFO:tensorflow:global_step/sec: 174.587
INFO:tensorflow:loss = 16.498056, step = 21200 (0.576 sec)
INFO:tensorflow:global_step/sec: 148.387
INFO:tensorflow:loss = 26.075485, step = 21300 (0.671 sec)
INFO:tensorflow:global_step/sec: 147.203
INFO:tensorflow:loss = 31.556446, step = 21400 (0.682 sec)
INFO:tensorflow:global_step/sec: 143.901
INFO:tensorflow:loss = 3.3695383, step = 21500 (0.707 sec)
INFO:tensorflow:global_step/sec: 157.153
INFO:tensorflow:loss = 14.648325, step = 21600 (0.620 sec)
INFO:tensorflow:global_step/sec: 182.23
IN

INFO:tensorflow:global_step/sec: 87.1868
INFO:tensorflow:loss = 17.190266, step = 28400 (1.141 sec)
INFO:tensorflow:global_step/sec: 155.634
INFO:tensorflow:loss = 19.21678, step = 28500 (0.646 sec)
INFO:tensorflow:global_step/sec: 189.526
INFO:tensorflow:loss = 15.864359, step = 28600 (0.516 sec)
INFO:tensorflow:global_step/sec: 95.8192
INFO:tensorflow:loss = 12.652697, step = 28700 (1.054 sec)
INFO:tensorflow:global_step/sec: 137.991
INFO:tensorflow:loss = 39.856735, step = 28800 (0.724 sec)
INFO:tensorflow:global_step/sec: 182.941
INFO:tensorflow:loss = 21.066622, step = 28900 (0.536 sec)
INFO:tensorflow:global_step/sec: 136.621
INFO:tensorflow:loss = 15.176967, step = 29000 (0.746 sec)
INFO:tensorflow:global_step/sec: 219.236
INFO:tensorflow:loss = 23.811766, step = 29100 (0.442 sec)
INFO:tensorflow:global_step/sec: 152.261
INFO:tensorflow:loss = 19.69292, step = 29200 (0.659 sec)
INFO:tensorflow:global_step/sec: 217.923
INFO:tensorflow:loss = 14.414072, step = 29300 (0.457 sec)
IN

INFO:tensorflow:loss = 4.5371804, step = 36600 (0.230 sec)
INFO:tensorflow:global_step/sec: 427.855
INFO:tensorflow:loss = 30.298962, step = 36700 (0.234 sec)
INFO:tensorflow:global_step/sec: 343.989
INFO:tensorflow:loss = 15.73698, step = 36800 (0.291 sec)
INFO:tensorflow:global_step/sec: 311.758
INFO:tensorflow:loss = 14.250308, step = 36900 (0.322 sec)
INFO:tensorflow:global_step/sec: 212.19
INFO:tensorflow:loss = 13.534255, step = 37000 (0.501 sec)
INFO:tensorflow:global_step/sec: 227.409
INFO:tensorflow:loss = 8.755979, step = 37100 (0.409 sec)
INFO:tensorflow:global_step/sec: 227.087
INFO:tensorflow:loss = 7.1454186, step = 37200 (0.442 sec)
INFO:tensorflow:global_step/sec: 277.82
INFO:tensorflow:loss = 54.1478, step = 37300 (0.359 sec)
INFO:tensorflow:global_step/sec: 340.349
INFO:tensorflow:loss = 12.523096, step = 37400 (0.293 sec)
INFO:tensorflow:global_step/sec: 297.913
INFO:tensorflow:loss = 26.37971, step = 37500 (0.340 sec)
INFO:tensorflow:global_step/sec: 249.683
INFO:te

In [26]:
eval_results = reloaded_regressor.evaluate(
    input_fn=lambda:eval_input_fn(df_test_norm, batch_size=8))

for key in eval_results:
    print('{:15s} {}'.format(key, eval_results[key]))
    
print('Average-Loss {:.4f}'.format(eval_results['average_loss']))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-11T17:01:10Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/autompg-dnnregressor/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-04-11-17:01:11
INFO:tensorflow:Saving dict for global step 40000: average_loss = 22.351118, global_step = 40000, label/mean 

In [27]:
pred_res = regressor.predict(input_fn=lambda: eval_input_fn(df_test_norm, batch_size=8))

print(next(iter(pred_res)))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/autompg-dnnregressor/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([15.416986], dtype=float32)}


# Boosted Tree Regressor

In [28]:
boosted_tree = tf.estimator.BoostedTreesRegressor(
    feature_columns=all_feature_columns,
    n_batches_per_layer=20,
    n_trees=200)

boosted_tree.train(
    input_fn=lambda:train_input_fn(df_train_norm, batch_size=BATCH_SIZE))

eval_results = boosted_tree.evaluate(
    input_fn=lambda:eval_input_fn(df_test_norm, batch_size=8))

print(eval_results)

print('Average-Loss {:.4f}'.format(eval_results['average_loss']))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/dx/yf09zs0954g6mbt7_q9bt9x80000gn/T/tmpha2xa7m6', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a43cf7710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflo

INFO:tensorflow:global_step/sec: 109.537
INFO:tensorflow:loss = 0.11498204, step = 5880 (0.725 sec)
INFO:tensorflow:global_step/sec: 131.909
INFO:tensorflow:loss = 0.26047596, step = 5980 (0.677 sec)
INFO:tensorflow:global_step/sec: 124.107
INFO:tensorflow:loss = 0.33735174, step = 6080 (0.858 sec)
INFO:tensorflow:global_step/sec: 145.937
INFO:tensorflow:loss = 0.1601444, step = 6180 (0.744 sec)
INFO:tensorflow:global_step/sec: 132.111
INFO:tensorflow:loss = 0.32895553, step = 6280 (0.556 sec)
INFO:tensorflow:global_step/sec: 177.313
INFO:tensorflow:loss = 0.10011798, step = 6380 (0.791 sec)
INFO:tensorflow:global_step/sec: 130.812
INFO:tensorflow:loss = 0.13596556, step = 6480 (0.462 sec)
INFO:tensorflow:global_step/sec: 200.829
INFO:tensorflow:loss = 0.23959085, step = 6580 (0.557 sec)
INFO:tensorflow:global_step/sec: 182.053
INFO:tensorflow:loss = 0.062182575, step = 6680 (0.422 sec)
INFO:tensorflow:global_step/sec: 251.761
INFO:tensorflow:loss = 0.104598515, step = 6780 (0.447 sec)

INFO:tensorflow:loss = 0.0020657773, step = 13980 (0.470 sec)
INFO:tensorflow:global_step/sec: 211.833
INFO:tensorflow:loss = 0.0059546297, step = 14080 (0.496 sec)
INFO:tensorflow:global_step/sec: 206.9
INFO:tensorflow:loss = 0.009486867, step = 14180 (0.488 sec)
INFO:tensorflow:global_step/sec: 204.524
INFO:tensorflow:loss = 0.0068497313, step = 14280 (0.484 sec)
INFO:tensorflow:global_step/sec: 208.102
INFO:tensorflow:loss = 0.004834181, step = 14380 (0.476 sec)
INFO:tensorflow:global_step/sec: 195.387
INFO:tensorflow:loss = 0.017496446, step = 14480 (0.488 sec)
INFO:tensorflow:global_step/sec: 211.382
INFO:tensorflow:loss = 0.008598793, step = 14580 (0.490 sec)
INFO:tensorflow:global_step/sec: 205.544
INFO:tensorflow:loss = 0.009535053, step = 14680 (0.486 sec)
INFO:tensorflow:global_step/sec: 211.36
INFO:tensorflow:loss = 0.005367078, step = 14780 (0.472 sec)
INFO:tensorflow:global_step/sec: 211.366
INFO:tensorflow:loss = 0.0037975176, step = 14880 (0.493 sec)
INFO:tensorflow:glob

INFO:tensorflow:loss = 0.000586186, step = 21980 (0.683 sec)
INFO:tensorflow:global_step/sec: 133.391
INFO:tensorflow:loss = 0.0005224318, step = 22080 (0.786 sec)
INFO:tensorflow:global_step/sec: 138.311
INFO:tensorflow:loss = 0.00059677864, step = 22180 (0.648 sec)
INFO:tensorflow:global_step/sec: 127.926
INFO:tensorflow:loss = 0.00068661605, step = 22280 (0.902 sec)
INFO:tensorflow:global_step/sec: 122.626
INFO:tensorflow:loss = 0.00054648146, step = 22380 (0.701 sec)
INFO:tensorflow:global_step/sec: 145.861
INFO:tensorflow:loss = 0.0012375885, step = 22480 (0.690 sec)
INFO:tensorflow:global_step/sec: 147.83
INFO:tensorflow:loss = 0.0007387131, step = 22580 (0.904 sec)
INFO:tensorflow:global_step/sec: 94.0449
INFO:tensorflow:loss = 0.0004914534, step = 22680 (1.249 sec)
INFO:tensorflow:global_step/sec: 86.3108
INFO:tensorflow:loss = 0.0020487844, step = 22780 (0.917 sec)
INFO:tensorflow:global_step/sec: 106.123
INFO:tensorflow:loss = 0.031996716, step = 22880 (0.783 sec)
INFO:tensor